# Preparation

## Import Library and Data

In [111]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [112]:
tourism_rating = pd.read_csv('tourism_rating.csv')
tourism_with_id = pd.read_csv('tourism_with_id.csv')

In [113]:
tourism_with_id.head()
tourism_rating.head()

,id_user,id_attraction,rating
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


In [114]:
tourism_with_id.isna().sum()

id_attraction            0
name_attraction          0
price_attraction         0
rating_avg_attraction    0
id_city                  0
id_attraction_cat        0
name_attraction_cat      0
name_city                0
desc_attraction          0
dtype: int64

In [115]:
tourism_with_id.drop(['rating_avg_attraction'],axis=1,inplace=True)
tourism_with_id

,id_attraction,name_attraction,price_attraction,id_city,id_attraction_cat,name_attraction_cat,name_city,desc_attraction
0,6,Taman Impian Jaya Ancol,25000,212,2,Taman Hiburan,Jakarta Utara,Taman Impian Jaya Ancol merupakan sebuah objek...
1,47,Taman Situ Lembang,0,216,2,Taman Hiburan,Jakarta Pusat,Taman Situ Lembang adalah sebuah taman kota ya...
2,66,Museum Layang-layang,10000,215,1,Budaya,Jakarta Selatan,Museum Layang-Layang adalah sebuah museum yang...
3,1,Monumen Nasional,20000,216,1,Budaya,Jakarta Pusat,Monumen Nasional atau yang populer disingkat d...
4,2,Kota Tua,0,213,1,Budaya,Jakarta Barat,"Kota tua di Jakarta, yang juga bernama Kota Tu..."
...,...,...,...,...,...,...,...,...
432,432,Taman Mundu,0,281,2,Taman Hiburan,Surabaya,Taman Mundu merupakan salah satu taman atau ru...
433,433,Museum Mpu Tantular,2000,281,1,Budaya,Surabaya,Museum Negeri Mpu Tantular adalah sebuah museu...
434,434,Taman Bungkul,0,281,2,Taman Hiburan,Surabaya,Taman Bungkul adalah taman wisata kota yang te...
435,435,Taman Air Mancur Menari Kenjeran,0,281,2,Taman Hiburan,Surabaya,Air mancur menari atau dancing fountain juga a...


In [116]:
def detect_outlier(data):
    outliers = []
    threeshold = 3
    mean = np.mean(data)
    std = np.std(data)

    for y in data:
        z_score = (y - mean)/std
        if np.abs(z_score) > threeshold:
            outliers.append(y)
    return outliers

In [117]:
tourism_with_id.isna().sum()
tourism_with_id = tourism_with_id.fillna(0)
print(tourism_with_id.isna().sum())
id_attraction = detect_outlier(tourism_with_id["id_attraction"])
print(id_attraction)
price_attraction = detect_outlier(tourism_with_id["price_attraction"])
print(price_attraction)
id_city = detect_outlier(tourism_with_id["id_city"])
print(id_city)
id_attraction_cat = detect_outlier(tourism_with_id["id_attraction_cat"])
print(id_attraction_cat)

id_attraction          0
name_attraction        0
price_attraction       0
id_city                0
id_attraction_cat      0
name_attraction_cat    0
name_city              0
desc_attraction        0
dtype: int64
[]
[270000, 900000, 300000, 250000, 500000, 280000, 375000]
[]
[]


## Modeling

In [118]:
user_ids = tourism_rating['id_user'].unique().tolist()
print('UserID list:', user_ids)

# Encode user IDs
user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}
print('Encoded UserID:', user_to_user_encoded)

# Decoding encoded user IDs
user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}
print('Decoded encoded User IDs:', user_encoded_to_user)

UserID list: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [119]:
# Convert placeID to a list without duplicate values
place_ids = tourism_rating['id_attraction'].unique().tolist()

# Encoding placeID
place_to_place_encoded = {x: i for i, x in enumerate(place_ids)}

# Decoding encoded values to placeID
place_encoded_to_place = {i: x for i, x in enumerate(place_ids)}

In [120]:
# Mapping userID to the user dataframe
tourism_rating['id_user'] = tourism_rating['id_user'].map(user_to_user_encoded)

# Mapping placeID to the place dataframe
tourism_rating['id_attraction'] = tourism_rating['id_attraction'].map(place_to_place_encoded)

In [121]:
# Getting the number of users
num_users = len(user_to_user_encoded)
print(num_users)

# Getting the number of places
num_place = len(place_encoded_to_place)
print(num_place)

# Converting the rating to float values
tourism_rating['rating'] = tourism_rating['rating'].values.astype(np.float32)

# Minimum rating value
min_rating = min(tourism_rating['rating'])

# Maximum rating value
max_rating = max(tourism_rating['rating'])

print('Number of User: {}, Number of place: {}, Min Rating: {}, Max Rating: {}'.format(
    num_users, num_place, min_rating, max_rating
))

300
437
Number of User: 300, Number of place: 437, Min Rating: 1.0, Max Rating: 5.0


In [122]:
collab_filtering = tourism_rating.sample(frac=1, random_state=42)
collab_filtering

,id_user,id_attraction,rating
6252,187,262,3.0
4684,141,83,2.0
1731,53,10,3.0
4742,143,141,3.0
4521,137,283,5.0
...,...,...,...
5734,172,175,5.0
5191,156,33,4.0
5390,162,52,3.0
860,27,187,4.0


In [123]:
x = collab_filtering[['id_user', 'id_attraction']].values

# Creating the variable y to represent the ratings
y = collab_filtering['rating'].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values

# Splitting the data into 80% for training and 20% for validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

print("Training data:")
print(x_train, y_train)
print("Validation data:")
print(x_val, y_val)

Training data:
[[118 288]
 [110 413]
 [169 215]
 ...
 [187 325]
 [273 272]
 [ 70 396]] [0.75 0.25 0.   ... 0.5  0.75 0.25]
Validation data:
[[216 212]
 [248 115]
 [139  75]
 ...
 [ 51  53]
 [ 82 330]
 [ 95 299]] [1.   0.   0.   ... 0.75 0.   0.5 ]


In [124]:
class RecommenderNet(tf.keras.Model):

  # Function Initialization
  def __init__(self, num_users, num_place, embedding_size, **kwargs):
    super(RecommenderNet, self).__init__(**kwargs)
    self.num_users = num_users
    self.num_place = num_place
    self.embedding_size = embedding_size
    self.user_embedding = layers.Embedding( # layer embedding user
        num_users,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.user_bias = layers.Embedding(num_users, 1) # layer embedding user bias
    self.place_embedding = layers.Embedding( # layer embeddings place
        num_place,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(1e-6)
    )
    self.place_bias = layers.Embedding(num_place, 1) # layer embedding place bias

  def call(self, inputs):
    user_vector = self.user_embedding(inputs[:, 0]) # calling embedding layer 1
    user_bias = self.user_bias(inputs[:, 0]) # calling embedding layer 2
    place_vector = self.place_embedding(inputs[:, 1]) # calling embedding layer 3
    place_bias = self.place_bias(inputs[:, 1]) # calling embedding layer 4

    dot_user_place = tf.tensordot(user_vector, place_vector, 2)

    x = dot_user_place + user_bias + place_bias

    return tf.nn.sigmoid(x) # sigmoid activation

In [125]:
 # Model Initialization
model = RecommenderNet(num_users, num_place, 50)

# Model Compile
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adagrad(learning_rate=0.05),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [126]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 8,
    epochs = 100,
    validation_data = (x_val, y_val)
)

Epoch 1/100
1000/1000 [==============================] - 3s 2ms/step - loss: 0.6949 - root_mean_squared_error: 0.3464 - val_loss: 0.6951 - val_root_mean_squared_error: 0.3471
Epoch 2/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6888 - root_mean_squared_error: 0.3420 - val_loss: 0.6947 - val_root_mean_squared_error: 0.3468
Epoch 3/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6849 - root_mean_squared_error: 0.3391 - val_loss: 0.6946 - val_root_mean_squared_error: 0.3467
Epoch 4/100
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6814 - root_mean_squared_error: 0.3366 - val_loss: 0.6948 - val_root_mean_squared_error: 0.3469
Epoch 5/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6788 - root_mean_squared_error: 0.3346 - val_loss: 0.6951 - val_root_mean_squared_error: 0.3471
Epoch 6/100
1000/1000 [==============================] - 2s 2ms/step - loss: 0.6760 - root_mean_squared_error: 0.3326 - val_l

In [127]:
place_df = tourism_with_id
df = tourism_rating


# User Sample
id_user = df.id_user.sample(1).iloc[0]
place_visited_by_user = df[df.id_user == id_user]

In [128]:
place_not_visited = place_df[~place_df['id_attraction'].isin(place_visited_by_user.id_attraction.values)]['id_attraction']
place_not_visited = list(
    set(place_not_visited)
    .intersection(set(place_to_place_encoded.keys()))
)

place_not_visited = [[place_to_place_encoded.get(x)] for x in place_not_visited]
user_encoder = user_to_user_encoded.get(id_user)
user_place_array = np.hstack(
    ([[user_encoder]] * len(place_not_visited), place_not_visited)
)

In [129]:
ratings = model.predict(user_place_array).flatten()

13/13 [==============================] - 0s 1ms/step


In [130]:
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_place_ids = [
    place_encoded_to_place.get(place_not_visited[x][0]) for x in top_ratings_indices
]

In [131]:
print('Showing recommendations for users: {}'.format(id_user))
print('place with high ratings from user')
print('----' * 8)

top_place_user = (
    place_visited_by_user.sort_values(
        by = 'rating',
        ascending=False
    )
    .head(5)
    .id_attraction.values
)

place_df_rows = place_df[place_df['id_attraction'].isin(top_place_user)]
for row in place_df_rows.itertuples():
    print(row.name_attraction)

print('----' * 8)
print('Top 10 place recommendation')
print('----' * 8)

recommended_place = place_df[place_df['id_attraction'].isin(recommended_place_ids)]
for row in recommended_place.itertuples():
    print(row.name_attraction)

Showing recommendations for users: 249
place with high ratings from user
--------------------------------
Istana Negara Republik Indonesia
Pantai Nguluran
Lawangwangi Creative Space
Situ Patenggang
Taman Buah Surabaya
--------------------------------
Top 10 place recommendation
--------------------------------
Monumen Nasional
Atlantis Water Adventure
Wisata Agro Edukatif Istana Susu Cibugary
Bukit Moko
Gereja Perawan Maria Tak Berdosa Surabaya
Monumen Tugu Pahlawan
Surabaya Museum (Gedung Siola)
Monumen Jalesveva Jayamahe
Waterpark Kenjeran Surabaya
Taman Hiburan Rakyat


## Save Model

In [132]:
# ## .H5

# model.save_weights("model.h5")

In [133]:
model.save('saved_model/my_model')

export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

In [134]:
export_dir = 'saved_model/1'

tf.saved_model.save(model,export_dir=export_dir)

In [135]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

## Download the Model

In [136]:
try:
    from google.colab import files
    files.download(tflite_model_file)
except:
    pass